https://github.com/facebookresearch/fairseq/tree/main/examples/roberta/commonsense_qa#3-evaluate

# NLP FS25 Course Project 2: Commensense Question Answering with Transformers

By David Hodel

Weighs & Biases Project: https://wandb.ai/dhodel-hslu-nlp/hslu-fs25-nlp-qa-transformers

## Introduction

In this notebook, I present my solution to the second course project of the FS25 NLP module at HSLU.

The task is to compare three Transformer models on the task of commonsense question answering:
1) A randomly initialized Transformer
2) A pre-trained Transformer (which was not trained / finetuned on CommonsenseQA)
3) An LLM (1B+ parameters) of my choice

I'll finetune the first two models and do prompt-engineering for the LLM. The goal is to compare the performance of these three models on the task of commonsense question answering.

### Dataset

We use the CommonsenseQA ([Talmor et al., 2019](https://aclanthology.org/N19-1421/)) dataset in this project. The dataset consists of 12,247 questions with 5 choices each, where only one is correct. The questions are designed to require commonsense reasoning to answer correctly.

The dataset was created by taking concepts from ConceptNet, a semantic network of commonsense knowledge.

## Setup

We first import the necessary libraries to run the code.

In [1]:
from collections import Counter
from datetime import datetime
import random
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torcheval.metrics as metrics
from torch.utils.data import Dataset

import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import AutoModelForCausalLM, pipeline

import wandb

We set up a fixed random seed to (at least try to) ensure reproducibility.

In [2]:
SEED = 42
seed_everything(SEED, workers=True)

Seed set to 42


42

Since we use Weights & Biases for experiment tracking, we first have to log in to our account.

In [4]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: davidhodel (waitless-hslu-dspro2-fs25) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
existing_sweep_id = None

### Data Splits

The data is available on Hugging Face: https://huggingface.co/datasets/tau/commonsense_qa.
Since only the train and validation splits have an answer key, we will use our own dataset splits.
We perform the splitting as presented in the lecture slides. We separate the last 1"000 samples from the training set as the validation set and use the original validation set as the test set.

In [5]:
train = load_dataset("tau/commonsense_qa", split="train[:-1000]")
valid = load_dataset("tau/commonsense_qa", split="train[-1000:]")
test = load_dataset("tau/commonsense_qa", split="validation")

print(len(train), len(valid), len(test))

8741 1000 1221


## Data Exploration

First, we want to take a look at the data to understand its structure and content.

In [ ]:
datasets = {
  "train": train,
  "validation": valid,
  "test": test
}

We ensure that all three splits have the same structure and that the answers are in the same format.

In [ ]:
print(train.column_names)
assert train.column_names == valid.column_names == test.column_names

print(train[0])

unique_answers = set([ex["answerKey"] for ex in train] + [ex["answerKey"] for ex in valid] + [ex["answerKey"] for ex in test])
print(f"Unique answer keys: {unique_answers}")

assert len(unique_answers) == 5

We then display a sample question and its answer for each split to get a feeling of the type of questions and answers.

In [ ]:
for split, data in datasets.items():
    print(f"\n=== {split} Split ===")
    print(f"Question: {data[0]['question']}")
    for j, choice in enumerate(data[0]['choices']['text']):
        print(f"{chr(65+j)}) {choice}")  # A, B, C, etc.
    print(f"Correct Answer: {data[0]['answerKey']}")
    print("=" * 50)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(14, 5))

labels = sorted(list(unique_answers))

colors = sns.color_palette("pastel")[0:5]

for i, (split, data) in enumerate(datasets.items()):
    answer_counts = Counter([ex["answerKey"] for ex in data])
    
    # Sort by labels to ensure consistent order
    counts = [answer_counts[label] for label in labels]
    
    ax[i].bar(labels, counts, color=colors)
    ax[i].set_xlabel("Answer Keys")
    ax[i].set_ylabel("Absolute Frequency")
    ax[i].set_title(f"{split.capitalize()} Set ({len(data)} samples)")
    
    # Add percentage annotations
    total = sum(counts)
    for j, count in enumerate(counts):
        percentage = count / total * 100
        ax[i].annotate(f"{percentage:.1f}%", 
                      xy=(labels[j], count),
                      xytext=(0, 3),
                      textcoords="offset points",
                      ha='center')

plt.suptitle("Distribution of Answer Keys Across Dataset Splits", fontsize=14)
plt.tight_layout()

We see that the distribution is relatively balanced, with a slight preference for answer `B` in the validation and test set.

We also plot the distribution of the number of characters in the questions.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 10))

colors = sns.color_palette("pastel")[0:3]
all_question_lengths = []

for i, (split, data) in enumerate(datasets.items()):
  question_lengths = [len(ex["question"]) for ex in data]
  all_question_lengths.append(question_lengths)
  
  # Histogram plots (top row)
  axes[0, i].hist(question_lengths, bins=30, color=colors[i])
  axes[0, i].set_xlabel("Question Length (characters)")
  axes[0, i].set_ylabel("Absolute Frequency")
  axes[0, i].set_title(f"{split.capitalize()} Question Length Distribution")
  
  # Add statistics as text
  axes[0, i].text(0.6, 0.95, 
      f"Min: {min(question_lengths)}\nMax: {max(question_lengths)}\nMean: {np.mean(question_lengths):.1f}\nMedian: {np.median(question_lengths)}",
      transform=axes[0, i].transAxes,
      bbox=dict(facecolor='white'),
      verticalalignment='top')
  
  # Boxplot (bottom row)
  axes[1, i].boxplot(question_lengths, patch_artist=True)
  axes[1, i].set_title(f"{split.capitalize()} Length Boxplot")
  axes[1, i].set_ylabel("Characters")
  
  # Set the boxplot fill color
  for patch in axes[1, i].get_children():
    if isinstance(patch, plt.matplotlib.patches.PathPatch):
      patch.set_facecolor(colors[i])

plt.suptitle("Distribution of Question Lengths Across Dataset Splits", fontsize=14)
plt.tight_layout()

We see that the questions are relatively short, with most of them having less than 100 characters. The three splits have a similar distribution and similar mean and median values.

The longest question has 376 characters which is good managable for a transformer model.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 10))

colors = sns.color_palette("pastel")[0:3]
all_choice_lengths = []

for i, (split, data) in enumerate(datasets.items()):
  choice_lengths = np.array([[len(choice) for choice in ex["choices"]["text"]] for ex in data]).flatten()
  all_choice_lengths.append(choice_lengths)
  
  # Histogram plots (top row)
  axes[0, i].hist(choice_lengths, bins=30, color=colors[i])
  axes[0, i].set_xlabel("Choice Length (characters)")
  axes[0, i].set_ylabel("Absolute Frequency")
  axes[0, i].set_title(f"{split.capitalize()} Choice Length Distribution")
  
  # Add statistics as text
  axes[0, i].text(0.6, 0.95, 
      f"Min: {min(choice_lengths)}\nMax: {max(choice_lengths)}\nMean: {np.mean(choice_lengths):.1f}\nMedian: {np.median(choice_lengths)}",
      transform=axes[0, i].transAxes,
      bbox=dict(facecolor='white'),
      verticalalignment='top')
  
  # Boxplot (bottom row)
  axes[1, i].boxplot(choice_lengths, patch_artist=True)
  axes[1, i].set_title(f"{split.capitalize()} Length Boxplot")
  axes[1, i].set_ylabel("Characters")
  
  # Set the boxplot fill color
  for patch in axes[1, i].get_children():
    if isinstance(patch, plt.matplotlib.patches.PathPatch):
      patch.set_facecolor(colors[i])

plt.suptitle("Distribution of Choice Lengths Across Dataset Splits", fontsize=14)
plt.tight_layout()

## Preprocessing

No real preprocessing is needed for the data.
The DistilRoBERTa model supports any english text, including lowercase / uppercase, punctuation, etc.

Same goes for the Phi-LLM, which also accepts any english text.

### Prepare Data for Tansformers

In [6]:
roberta_tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base", use_fast=False) # disable fast tokenizer for multi-threaded tokenization (https://stackoverflow.com/a/72926996)

In [7]:
encoded = roberta_tokenizer.encode("Is this working?", return_tensors="pt")
decoded = roberta_tokenizer.decode(encoded[0])

print(f"Encoded: {encoded}")
print(f"Decoded: {decoded}")

Encoded: tensor([[   0, 6209,   42,  447,  116,    2]])
Decoded: <s>Is this working?</s>


In [8]:
def answer_key_to_index(answer_key):
  return ord(answer_key) - ord("A")

def index_to_answer_key(index):
  return chr(index + ord("A"))

assert answer_key_to_index("A") == 0
assert index_to_answer_key(0) == "A"

In [9]:
class CommonsenseQADataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length, debug=False):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.debug = debug
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        example = self.dataset[idx]
        question = example["question"]
        choices = example["choices"]["text"]
        
        answer_index = answer_key_to_index(example["answerKey"])
            
        # Tokenize all question-answer pairs but don't pad yet
        encodings = []
        for choice in choices:
            encoding = self.tokenizer(
                question,
                choice,
                truncation=False,
                return_tensors=None  # Return lists, not tensors
            )

            if self.debug:
                # Assert that max_length is respected
                assert len(encoding["input_ids"]) <= self.max_length, "Input exceeds max length"

            encodings.append(encoding)
            
        return encodings, answer_index

In [10]:
class MultipleChoiceCollator:
    def __init__(self, tokenizer, debug=False):
        self.tokenizer = tokenizer
        self.debug = debug
        
    def __call__(self, batch):
        # Unpack the batch - each item is now a tuple of (encodings, label)
        encodings_list = [item[0] for item in batch]  # List of lists of encodings
        labels = [item[1] for item in batch]  # List of labels
        
        # Flatten all encodings
        flat_encodings = [encoding for encodings in encodings_list for encoding in encodings]
        
        # Pad to the longest in this batch
        padded_encodings = self.tokenizer.pad(
            flat_encodings,
            padding=True,
            return_tensors="pt"
        )

        num_choices = 5
        batch_size = len(batch)

        # Reshape back to [batch_size, num_choices, seq_length]
        input_ids = padded_encodings["input_ids"].view(batch_size, num_choices, -1)
        attention_mask = padded_encodings["attention_mask"].view(batch_size, num_choices, -1)
        
        # Convert labels to tensor
        labels = torch.tensor(labels, dtype=torch.long)
        
        # Return a tuple of (input_ids, attention_mask, labels)
        return input_ids, attention_mask, labels

In [11]:
class CommonsenseQADataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, valid_dataset, test_dataset, tokenizer, batch_size=16, max_length=512, num_workers=8, debug=False):
        super().__init__()
        self.train_dataset = train_dataset
        self.valid_dataset = valid_dataset
        self.test_dataset = test_dataset
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_length = max_length
        self.num_workers = num_workers
        self.debug = debug
        
    def setup(self, stage=None):
        # Create datasets
        if stage == 'fit' or stage is None:
            self.train_ds = CommonsenseQADataset(self.train_dataset, self.tokenizer, self.max_length, debug=self.debug)
            self.val_ds = CommonsenseQADataset(self.valid_dataset, self.tokenizer, self.max_length, debug=self.debug)
            
            if self.debug:
                # Ensure datasets have expected properties
                assert len(self.train_ds) == len(self.train_dataset), "Train dataset length mismatch"
                assert len(self.val_ds) == len(self.valid_dataset), "Validation dataset length mismatch"
        
        if stage == 'test' or stage is None:
            self.test_ds = CommonsenseQADataset(self.test_dataset, self.tokenizer, self.max_length, debug=self.debug)

            if self.debug:
                assert len(self.test_ds) == len(self.test_dataset), "Test dataset length mismatch"
    
    def train_dataloader(self):
        loader = torch.utils.data.DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            collate_fn=MultipleChoiceCollator(self.tokenizer, debug=self.debug),
            shuffle=True,
            num_workers=self.num_workers
        )
        return loader
    
    def val_dataloader(self):
        loader = torch.utils.data.DataLoader(
            self.val_ds,
            batch_size=self.batch_size,
            collate_fn=MultipleChoiceCollator(self.tokenizer, debug=self.debug),
            shuffle=False,
            num_workers=self.num_workers
        )
        return loader
    
    def test_dataloader(self):
        loader = torch.utils.data.DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            collate_fn=MultipleChoiceCollator(self.tokenizer, debug=self.debug),
            shuffle=False,
            num_workers=self.num_workers
        )
        return loader

In [12]:
# Initialize DataModule
max_input_length = 512 # 514 (as specified in config.json of distillroberta-base model) - 2 (for [CLS] and [SEP]) 
data_module = CommonsenseQADataModule(train, valid, test, roberta_tokenizer, batch_size=24, max_length=max_input_length, debug=False)

### Prepare Data for LLM

In [ ]:
class PhiPromptDataset(Dataset):
    def __init__(self, dataset, random_subset_size=1.0):
        self.dataset = dataset
        self.prompts = []
        self.correct_answers = []

        self.prompt_template = """<|system|>You are a helpful assistant in a multiple-choice question-answering task. Answer the following multiple-choice commonsense reasoning question with just the letter of the correct answer (A, B, C, D, or E). Do not provide any explanations or additional information. Your response must not contain the full answer, only the letter.<|end|>
<|user|>Question: What do students do in school?
Choices:
A They play outside.
B They eat lunch.
C They go home.
D They learn and study.
E They sleep.<|end|>
<|assistant|>D<|end|>
<|user|>Question: If you leave ice out in the sun, what will most likely happen to it?
Choices:
A It will catch fire
B It will melt
C It will grow bigger
D It will turn into dust
E It will start glowing<|end|>
<|assistant|>B<|end|>
<|user|>Question: If you are hungry, what is the most logical thing to do?
Choices:
A Take a nap
B Go for a swim
C Eat some food
D Buy new shoes
E Read a book<|end|>
<|assistant|>C<|end|>
<|user|>Question: {question}
Choices:
A {choice_a}
B {choice_b}
C {choice_c}
D {choice_d}
E {choice_e}<|end|>
<|assistant|>"""

        
        self.prepare_data(random_subset_size)
    
    def prepare_data(self, random_subset_size=1.0):
        # If random_subset_size is 1.0, use the entire dataset
        if random_subset_size >= 1.0:
            subset = self.dataset
        else:
            # Calculate the number of examples to include
            subset_size = max(1, int(len(self.dataset) * random_subset_size))
            
            # Get random indices without replacement
            indices = random.sample(range(len(self.dataset)), subset_size)
            
            # Create the subset
            subset = [self.dataset[i] for i in indices]
        
        # Process the subset
        for example in subset:
            question = example["question"]
            choices = example["choices"]["text"]
            correct_answer = answer_key_to_index(example["answerKey"])
            prompt = self.create_prompt(question, choices)
            
            self.prompts.append(prompt)
            self.correct_answers.append(correct_answer)
    
    def create_prompt(self, question, choices):
        prompt = self.prompt_template.format(
            question=question,
            choice_a=choices[0],
            choice_b=choices[1],
            choice_c=choices[2],
            choice_d=choices[3],
            choice_e=choices[4]
        )
        
        return prompt
    
    def __len__(self):
        return len(self.prompts)
    
    def __getitem__(self, idx):
        return {
            "prompt": self.prompts[idx],
            "correct_answer": self.correct_answers[idx]
        }

In [ ]:
phi_prompt_valid = PhiPromptDataset(valid, random_subset_size=0.1) # around 100 samples is usually enough to see the model's performance
phi_prompt_test = PhiPromptDataset(test)

phi_prompt_valid[0], phi_prompt_test[0]

## Models

### 1) Pre-trained Transformer

I decided to use a distilled version of the [RoBERTa base model](https://huggingface.co/FacebookAI/roberta-base) for this task. The model is available on Hugging Face ([distilbert/distilroberta-base](https://huggingface.co/distilbert/distilroberta-base)) and was trained using the same procedure as DistilBERT.

The model has 6 layers, 768 dimension and 12 heads, totalizing 82M parameters (compared to 125M parameters for RoBERTa-base). According to Hugging Face, the model runs on average twice as fast as Roberta-base.

In [13]:
pretrained_distilroberta = AutoModel.from_pretrained("distilbert/distilroberta-base")

In [14]:
class RobertaMultipleChoiceModel(pl.LightningModule):
    def __init__(self, roberta_model, dropout_prob=0.1, learning_rate=1e-5, weight_decay=1e-3, use_layer_norm=True, hidden_size_multiplier=1.0, debug=False):
        super().__init__()
        
        self.save_hyperparameters(ignore=['roberta_model'])

        self.roberta = roberta_model
        self.roberta.train()

        hidden_size = int(hidden_size_multiplier * self.roberta.config.hidden_size)
        print(f"Hidden size: {hidden_size}")
        
        # Custom classification head
        self.dropout = nn.Dropout(dropout_prob)
        self.classifier = nn.Sequential(
            nn.Linear(self.roberta.config.hidden_size, hidden_size),
            nn.LayerNorm(hidden_size) if use_layer_norm else nn.Identity(),
            nn.ReLU(), # non-linearity
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_size, 1) # single score per candidate
        )
        
        # Loss function
        self.criterion = nn.CrossEntropyLoss()
        
        # Metrics
        self.train_accuracy = metrics.MulticlassAccuracy(num_classes=5)
        self.val_accuracy = metrics.MulticlassAccuracy(num_classes=5)
        self.test_accuracy = metrics.MulticlassAccuracy(num_classes=5)

        self.test_y = []
        self.test_y_pred = []

        self.debug = debug
    
    def forward(self, input_ids, attention_mask):
        # input_ids and attention_mask have shape: [batch_size, num_choices, seq_length]
        this_batch_size, num_choices, seq_length = input_ids.shape

        if self.debug:
            assert num_choices == 5, "Number of choices should be 5 for CommonsenseQA"
            assert seq_length <= self.roberta.config.max_position_embeddings, "Sequence length exceeds model's max position embeddings"
            assert input_ids.shape == attention_mask.shape, "Input IDs and attention mask should have the same shape"
        
        # Reshape to feed through the model
        input_ids = input_ids.view(-1, seq_length)  # [batch_size * num_choices, seq_length]
        attention_mask = attention_mask.view(-1, seq_length)  # [batch_size * num_choices, seq_length]

        if self.debug:
            assert input_ids.shape == attention_mask.shape, "Input IDs and attention mask should have the same shape"
            assert input_ids.shape[0] == attention_mask.shape[0] == this_batch_size * num_choices, "First dimension should be batch size * num choices"
            assert input_ids.shape[1] == attention_mask.shape[1] == seq_length, "Second dimension should be sequence length"
        
        # Forward pass through base model
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        
        # Get the first token (<s>) representation
        pooled_output = outputs.last_hidden_state[:, 0]  # [batch_size * num_choices, hidden_size]
        pooled_output = self.dropout(pooled_output)

        if self.debug:
            assert pooled_output.shape == (this_batch_size * num_choices, self.roberta.config.hidden_size), "Pooled output should have shape [batch_size * num_choices, hidden_size]"
        
        # Get logits for each choice
        logits = self.classifier(pooled_output)  # [batch_size * num_choices, 1]

        if self.debug:
            assert logits.shape == (this_batch_size * num_choices, 1), "Logits should have shape [batch_size * num_choices, 1]"
        
        # Reshape logits back to [batch_size, num_choices]
        reshaped_logits = logits.view(this_batch_size, num_choices)

        if self.debug:
            assert reshaped_logits.shape == (this_batch_size, num_choices), "Reshaped logits should have shape [batch_size, num_choices]"
        
        return reshaped_logits
    
    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        logits = self(input_ids, attention_mask)
        loss = self.criterion(logits, labels)
        
        # Update metrics
        self.train_accuracy.update(logits, labels)
        
        # Log metrics
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_acc", self.train_accuracy.compute().item(), on_step=False, on_epoch=True, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        logits = self(input_ids, attention_mask)
        loss = self.criterion(logits, labels)
        
        # Update metrics
        self.val_accuracy.update(logits, labels)
        
        # Log metrics
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", self.val_accuracy.compute().item(), on_epoch=True, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        logits = self(input_ids, attention_mask)
        loss = self.criterion(logits, labels)

        # Update metrics
        self.test_accuracy.update(logits, labels)

        # Log metrics
        self.log("test_loss", loss, on_epoch=True)
        self.log("test_acc", self.test_accuracy.compute().item(), on_epoch=True)

        self.test_y_pred.extend(logits.argmax(dim=1).cpu().numpy())
        self.test_y.extend(labels.cpu().numpy())

        return loss
    
    def reset_test_arrays(self):
        self.test_y = []
        self.test_y_pred = []

    def configure_optimizers(self):
        # Group parameters to apply a lower learning rate to the transformer layers
        transformer_lr_multiplier = 0.05
        transformer_lr = transformer_lr_multiplier * self.hparams.learning_rate
        classifier_lr = self.hparams.learning_rate

        optimizer_grouped_parameters = [
            {
                "params": [p for _, p in self.roberta.named_parameters()],
                "lr": transformer_lr,
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for _, p in self.classifier.named_parameters()],
                "lr": classifier_lr,
                "weight_decay": self.hparams.weight_decay,
            },
        ]

        optimizer = torch.optim.AdamW(optimizer_grouped_parameters)
        
        # Set up learning rate scheduler
        total_steps = self.trainer.estimated_stepping_batches
        warmup_steps = total_steps // 10
        
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=[transformer_lr, classifier_lr],  # Specify max_lr for each group
            total_steps=total_steps,
            pct_start=warmup_steps / total_steps,
            div_factor=100,
            final_div_factor=1000,
            anneal_strategy="linear"
        )
        
        scheduler_config = {
            "scheduler": scheduler,
            "interval": "step",
            "frequency": 1,
        }
        
        return [optimizer], [scheduler_config]

### 2) Randomly Initialized Transformer

In [15]:
# Create a randomly initialized model using the same configuration
random_config = AutoConfig.from_pretrained("distilbert/distilroberta-base")
random_initialized_roberta = AutoModel.from_config(random_config)

### 3) LLM - Phi-4-mini

For the LLM approach, I'll use Phi-4-mini-instruct, which is a 3.8 billion parameter model released by Microsoft in February 2025. The `-instruct` version of the model is designed to follow instructions and answer questions in a conversational manner. The model is available on Hugging Face ([microsoft/Phi-4-mini-instruct](https://huggingface.co/microsoft/Phi-4-mini-instruct)).

According to Microsoft, the model is "built upon synthetic data and filtered publicly available websites - with a focus on high-quality, reasoning dense data" ([source](https://ollama.com/library/phi4-mini)).



Unlike the previous models in this notebook, this model won't be fine-tuned but will use prompt engineering techniques.

In [ ]:
phi_model_name = "microsoft/Phi-4-mini-instruct"

phi_tokenizer = AutoTokenizer.from_pretrained(phi_model_name, fast=False) # disable fast tokenizer for multi-threaded tokenization (https://stackoverflow.com/a/72926996)

phi_model = AutoModelForCausalLM.from_pretrained(
    phi_model_name, 
    torch_dtype="auto",  
    trust_remote_code=True,
)

In [ ]:
class PhiPromptEngineering:
    def __init__(self, model, tokenizer, debug=False):
        self.model = model
        self.debug = debug

        self.pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=1024,
            do_sample=False, # we want deterministic output
            return_full_text=False # only return the newly generated text
        )
    
    def predict(self, prompt):
        full_response = self.pipe(prompt)[0]["generated_text"].strip()
        
        if self.debug:
            print(f"Full response: '{full_response}'")
        
        # Extract just the final letter
        # Method 1: Get the very last character if it's a valid choice
        final_char = full_response[-1]
        if final_char in "ABCDE":
            answer = final_char
        # Method 2: More robust - find the last occurrence of A, B, C, D, or E
        else:
            for char in reversed(full_response):
                if char in "ABCDE":
                    answer = char
                    break
            else:  # No valid choice found
                raise ValueError(f"No valid answer choice (A-E) found in response: '{full_response}'")
        
        if self.debug:
            print(f"Extracted answer: '{answer}'")
        
        assert answer in "ABCDE", f"Extracted answer '{answer}' is not a valid choice (A-E)"
        
        return answer_key_to_index(answer.upper())

    
    def evaluate(self, dataset, log_wandb=True):
        correct = 0
        total = 0
        accuracy = 0.0

        y = []
        y_pred = []
        
        samples_count = len(dataset)
        for i in (pbar := trange(samples_count)):
            pbar.set_description(f"Sample {i}/{samples_count}")

            sample = dataset[i]
            prompt, correct_answer = sample["prompt"], sample["correct_answer"]
            predicted_index = self.predict(prompt)

            y.append(correct_answer)
            y_pred.append(predicted_index)
            
            total += 1
            if predicted_index == correct_answer:
                correct += 1

            accuracy = correct / total
            pbar.set_postfix({"accuracy": accuracy})

            if log_wandb:
                wandb.log({
                    "accuracy": accuracy
                })

        return accuracy, y, y_pred

## Training

### Hyperparameter Optimization

In [22]:
def train(model, data_module, max_epochs, checkpoints_path, wandb_run_prefix=None, early_stopping_patience=None, debug=False, existing_run=None):
  run = existing_run if existing_run else wandb.init(entity="dhodel-hslu-nlp", project="hslu-fs25-nlp-qa-transformers", name=f"{wandb_run_prefix}-{datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}")

  seed_everything(SEED, workers=True)

  if early_stopping_patience is None:
    early_stopping_patience = max_epochs + 1 # disable early stopping

  if debug:
    max_epochs = 1

  data_module.setup("fit")

  best_checkpoint_callback = ModelCheckpoint(
      dirpath=checkpoints_path,
      filename="best-{epoch:02d}-{val_acc:.4f}",
      save_top_k=1,
      monitor="val_acc",
      mode="max"
  )

  regular_checkpoint_callback = ModelCheckpoint(
      dirpath=checkpoints_path,
      filename="latest-{epoch:02d}",
      save_top_k=1, # only keep the most recent checkpoint
      every_n_epochs=1, # save every epoch
  )

  early_stop_callback = EarlyStopping(
    monitor="val_acc",
    patience=early_stopping_patience,
    mode="max",
    check_finite=True # check for NaN or inf values
  )
  
  lr_callback = LearningRateMonitor()

  wandb_logger = WandbLogger(
    experiment=run,
    log_model=(not debug)
  )

  torch.set_float32_matmul_precision('high')
  trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator="auto", # Uses GPU if available, otherwise CPU
    callbacks=[best_checkpoint_callback, regular_checkpoint_callback, early_stop_callback, lr_callback],
    logger=wandb_logger,
    log_every_n_steps=10,
  )

  trainer.fit(model, data_module)

  return trainer, best_checkpoint_callback.best_model_path, run

In [23]:
# Define the sweep configuration for hyperparameter optimization
sweep_config = {
    'method': 'bayes',
    'name': f'roberta-hyperparam-sweep-{datetime.now().strftime("%Y-%m-%dT%H:%M:%S")}',
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'values': [3e-6, 5e-6, 1e-5, 3e-5, 1e-4, 3e-4]
        },
        'weight_decay': {
            'values': [1e-5, 1e-4, 1e-3]
        },
        'dropout_prob': {
            'values': [0.1, 0.2, 0.3, 0.4]
        },
        'use_layer_norm': {
            'values': [True, False]
        },
        'hidden_size_multiplier': {
            'values': [0.5, 1.0, 1.5]
        }
    }
}

In [24]:
def run_sweep_trial():
    with wandb.init() as run:
        run.name = f"pretrained-roberta-sweep-{wandb.run.sweep_id}-{datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}"
        config = wandb.config

        # Create a unique checkpoint directory for this sweep run
        sweep_checkpoints_path = f"./checkpoints/sweep/{wandb.run.id}"
        os.makedirs(sweep_checkpoints_path, exist_ok=True)
        
        # Create the model with hyperparameters from the sweep
        model = RobertaMultipleChoiceModel(
            roberta_model=pretrained_distilroberta,
            dropout_prob=config.dropout_prob,
            learning_rate=config.learning_rate,
            weight_decay=config.weight_decay,
            use_layer_norm=config.use_layer_norm,
            hidden_size_multiplier=config.hidden_size_multiplier,
            debug=False
        )
        
        # Use the existing train function
        _, best_checkpoint, _ = train(
            model=model,
            data_module=data_module,
            max_epochs=50,
            checkpoints_path=sweep_checkpoints_path,
            early_stopping_patience=5,
            debug=False,
            existing_run=run
        )
        
        # Save the best checkpoint path for later reference
        with open(f"{sweep_checkpoints_path}/best_checkpoint.txt", "w") as f:
            f.write(best_checkpoint)

        run.finish()

In [20]:
# Initialize the sweep
if existing_sweep_id:
  # no need to create a new sweep, just use the existing one
  sweep_id = existing_sweep_id
else:
  sweep_id = wandb.sweep(sweep_config, entity="dhodel-hslu-nlp", project="hslu-fs25-nlp-qa-transforme rs")

Create sweep with ID: b89yrm6b
Sweep URL: https://wandb.ai/dhodel-hslu-nlp/hslu-fs25-nlp-qa-transformers/sweeps/b89yrm6b


In [25]:
if existing_sweep_id:
  print(f"Using existing sweep ID: {existing_sweep_id}")
else:
  wandb.agent(sweep_id, function=run_sweep_trial, count=50)

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 6lbkb493 with config:
wandb: 	dropout_prob: 0.2
wandb: 	hidden_size_multiplier: 1
wandb: 	learning_rate: 1e-05
wandb: 	use_layer_norm: True
wandb: 	weight_decay: 0.001


Seed set to 42


Hidden size: 768


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name       | Type             | Params | Mode 
--------------------------------------------------------
0 | roberta    | RobertaModel     | 82.1 M | train
1 | dropout    | Dropout          | 0      | train
2 | classifier | Sequential       | 592 K  | train
3 | criterion  | CrossEntropyLoss | 0      | train
--------------------------------------------------------
82.7 M    Trainable params
0         Non-trainable params
82.7 M    Total params
330.845   Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode
wandb: WARNING Config item 'dropout_prob' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was l

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

wandb: Ctrl + C detected. Stopping sweep.


Validation: |          | 0/? [00:00<?, ?it/s]

### Get Best Hyperparameters

In [26]:
# Get the best hyperparameters from the sweep
api = wandb.Api()
sweep = api.sweep(f"dhodel-hslu-nlp/hslu-fs25-nlp-qa-transformers/{sweep_id}")
best_run = sweep.best_run()

# Get the best hyperparameters
best_config = best_run.config
print("Best hyperparameters:")
print(f"Learning Rate: {best_config['learning_rate']}")
print(f"Weight Decay: {best_config['weight_decay']}")
print(f"Dropout Probability: {best_config['dropout_prob']}")
print(f"Use Layer Norm: {best_config['use_layer_norm']}")
print(f"Hidden Size Multiplier: {best_config['hidden_size_multiplier']}")

Traceback (most recent call last):
wandb: ERROR Problem finishing run
Exception in thread Thread-35:
Traceback (most recent call last):
  File "/tmp/ipykernel_25416/4280763003.py", line 22, in run_sweep_trial
  File "/tmp/ipykernel_25416/2469710868.py", line 52, in train
  File "/home/david/miniconda3/envs/nlp-real/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 561, in fit
    call._call_and_handle_interrupt(
  File "/home/david/miniconda3/envs/nlp-real/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/home/david/miniconda3/envs/nlp-real/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/home/david/miniconda3/envs/nlp-real/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 1012, in _run
    results = self._run_stage()
  File "/home/david/miniconda3/envs

wandb: Sorting runs by -summary_metrics.val_acc


Best hyperparameters:
Learning Rate: 1e-05
Weight Decay: 0.001
Dropout Probability: 0.2
Use Layer Norm: True
Hidden Size Multiplier: 1


### 1) Pre-trained Transformer with Best Hyperparameters

In [27]:
# Initialize models with the best hyperparameters
pretrained_roberta_model = RobertaMultipleChoiceModel(
    roberta_model=pretrained_distilroberta, 
    dropout_prob=best_config['dropout_prob'], 
    learning_rate=best_config['learning_rate'], 
    weight_decay=best_config['weight_decay'], 
    use_layer_norm=best_config['use_layer_norm'], 
    hidden_size_multiplier=best_config['hidden_size_multiplier'],
    debug=False
)

Hidden size: 768


In [ ]:
pretrained_checkpoints_path = "./checkpoints/pretrained"
os.makedirs(pretrained_checkpoints_path, exist_ok=True)

pretrained_roberta_trainer, pretrained_roberta_best_checkpoint = train(
  model=pretrained_roberta_model,
  data_module=data_module,
  max_epochs=50,
  checkpoints_path=pretrained_checkpoints_path,
  early_stopping_patience=5,
  wandb_run_prefix="pretrained-roberta-best-params",
  debug=False
)

### 2) Randomly Initialized Transformer with Best Hyperparameters

In [ ]:
# Initialize the randomly initialized model with the same best hyperparameters
random_initialized_roberta_model = RobertaMultipleChoiceModel(
    roberta_model=random_initialized_roberta, 
    dropout_prob=best_config['dropout_prob'], 
    learning_rate=best_config['learning_rate'], 
    weight_decay=best_config['weight_decay'], 
    use_layer_norm=best_config['use_layer_norm'], 
    hidden_size_multiplier=best_config['hidden_size_multiplier'],
    debug=False
)

In [ ]:
random_initialized_checkpoints_path = "./checkpoints/random-initialized"
os.makedirs(random_initialized_checkpoints_path, exist_ok=True)

randomly_initialized_roberta_trainer, randomly_initialized_roberta_best_checkpoint = train(
  model=random_initialized_roberta_model,
  data_module=data_module,
  max_epochs=50,
  checkpoints_path=random_initialized_checkpoints_path,
  early_stopping_patience=5,
  wandb_run_prefix="random-initialized-roberta-best-params",
)

### 3) Prompt Engineering with Phi LLM

For the Phi model, we don't need training since we're using prompt engineering.

For trying out different prompt, I run the mode against a subset of the validation set as the trend of the performance emerges quickly.

What I've tried:
- Give between 3 and 6 example questions and answers
- Include easier and harder examples
- Add clarity about tie-breaking: `If multiple options seem correct, choose the MOST appropriate or complete answer`
- Chain-of-thought prompting
- Encourage double-checking: `Before submitting your answer, verify it makes logical sense and is the most appropriate choice for the question.`

In conclusion, the simple prompt with just 2 examples to tune it to the task seems to work best.
All othger additions made only a small difference, mostly negative.
The chain-of-thought prompting made the evaluating much slower (factor ~50) due to the increased number of tokens. 

In [ ]:
phi_prompt_engineering = PhiPromptEngineering(
    model=phi_model,
    tokenizer=phi_tokenizer,
    debug=False
)

In [ ]:
wandb.init(
    entity="dhodel-hslu-nlp",
    project="hslu-fs25-nlp-qa-transformers",
    name=f"phi-prompt-engineering-{datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}",
    reinit=True,
    config={
        "model_name": phi_model_name,
        "prompt_template": phi_prompt_valid.prompt_template,
    }
)

In [ ]:
phi_accuracy, y, y_pred = phi_prompt_engineering.evaluate(phi_prompt_valid, log_wandb=True)

In [ ]:
wandb.finish()

## Evaluation

In [ ]:
pretrained_roberta_trainer.logger = False
randomly_initialized_roberta_trainer.logger = False

In [ ]:
pretrained_roberta_model.reset_test_arrays()
pretrained_test_results = pretrained_roberta_trainer.test(pretrained_roberta_model, datamodule=data_module, ckpt_path=pretrained_roberta_best_checkpoint)

random_initialized_roberta_model.reset_test_arrays()
random_test_results = randomly_initialized_roberta_trainer.test(random_initialized_roberta_model, datamodule=data_module, ckpt_path=randomly_initialized_roberta_best_checkpoint)

pretrained_test_labels = pretrained_roberta_model.test_y
pretrained_test_preds = pretrained_roberta_model.test_y_pred

random_test_labels = random_initialized_roberta_model.test_y
random_test_preds = random_initialized_roberta_model.test_y_pred

In [ ]:
phi_accuracy, phi_test_labels, phi_test_preds = phi_prompt_engineering.evaluate(phi_prompt_test, log_wandb=False)

In [ ]:
label_mapping = {i: chr(65 + i) for i in range(5)}  # 0->A, 1->B, etc.
label_names = list(label_mapping.values())

pretrained_cm = confusion_matrix(pretrained_test_labels, pretrained_test_preds)
random_cm = confusion_matrix(random_test_labels, random_test_preds)
phi_cm = confusion_matrix(phi_test_labels, phi_test_preds)

# Determine the global min and max values for consistent scaling
global_vmin = min(pretrained_cm.min(), random_cm.min(), phi_cm.min())
global_vmax = max(pretrained_cm.max(), random_cm.max(), phi_cm.max())

# Create a figure with three subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21, 6))

# Plot confusion matrices
sns.heatmap(pretrained_cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=label_names, yticklabels=label_names, 
            ax=ax1, vmin=global_vmin, vmax=global_vmax)
ax1.set_title("Pretrained RoBERTa Model Confusion Matrix", fontsize=14)
ax1.set_xlabel("Predicted Choice", fontsize=12)
ax1.set_ylabel("True Choice", fontsize=12)

sns.heatmap(random_cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=label_names, yticklabels=label_names, 
            ax=ax2, vmin=global_vmin, vmax=global_vmax)
ax2.set_title("Random Initialized RoBERTa Model Confusion Matrix", fontsize=14)
ax2.set_xlabel("Predicted Choice", fontsize=12)
ax2.set_ylabel("True Choice", fontsize=12)

sns.heatmap(phi_cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=label_names, yticklabels=label_names, 
            ax=ax3, vmin=global_vmin, vmax=global_vmax)
ax3.set_title("Phi-4-mini Confusion Matrix", fontsize=14)
ax3.set_xlabel("Predicted Choice", fontsize=12)
ax3.set_ylabel("True Choice", fontsize=12)

plt.tight_layout()
plt.savefig("graphics/confusion_matrices_all_models.svg", bbox_inches="tight")
plt.show()

# Create a bar chart to compare model performance
model_names = ['Pretrained RoBERTa', 'Random Initialized RoBERTa', 'Phi-4-mini']
accuracies = [
    pretrained_test_results[0]['test_acc'],
    random_test_results[0]['test_acc'],
    phi_accuracy
]

plt.figure(figsize=(10, 6))
bars = plt.bar(model_names, accuracies, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.ylim(0, 1.0)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Model Performance Comparison', fontsize=14)
plt.xticks(rotation=15, ha='right')

# Add the accuracy values on top of the bars
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2., 
             bar.get_height() + 0.01, 
             f'{acc:.4f}', 
             ha='center')

plt.tight_layout()
plt.savefig('graphics/model_comparison.svg', bbox_inches='tight')
plt.show()

## Interpretation

### Interpretation of Results

The comparison between our three models - pre-trained RoBERTa, randomly initialized RoBERTa, and Phi-4-mini LLM (with prompt engineering) - reveals several interesting patterns in their performance on the CommonsenseQA task.

1. **Pre-trained RoBERTa**: The model leverages transfer learning from its pre-training phase, giving it a strong foundation for understanding language patterns and semantics before fine-tuning on our specific task.

2. **Randomly Initialized RoBERTa**: Starting from scratch, this model had to learn language patterns solely from our training data, which is much more challenging given the limited size of the dataset compared to typical pre-training datasets.

3. **Phi-4-mini LLM (Prompt Engineering)**: This approach uses a much larger model (7B parameters) without any task-specific fine-tuning, relying instead on prompt engineering to elicit the desired behavior.

The confusion matrices reveal each model's specific strengths and weaknesses in predicting different answer choices. The bar chart provides a clear comparison of overall accuracy between the three approaches.

### Key Observations

- **Effect of Pre-training**: The significant performance gap between the pre-trained and randomly initialized models demonstrates the value of transfer learning, especially for tasks with limited labeled data.

- **LLM with Prompt Engineering**: Phi's performance shows how effectively large language models can be adapted to specific tasks without fine-tuning, using only careful prompt design.

- **Error Patterns**: The confusion matrices show different patterns of errors across models, suggesting they may be making different types of mistakes despite being evaluated on the same task.

### Conclusions

This comparison illustrates the trade-offs between different approaches to transformer-based question answering:

- Pre-trained + fine-tuned models offer strong performance with reasonable computational requirements
- Randomly initialized models struggle without transfer learning benefits
- Large LLMs with prompt engineering can achieve competitive results without task-specific training, but at higher computational cost

The results highlight how different transformer-based approaches can be selected based on available resources, performance requirements, and deployment constraints.

## Tools Used

- Visual Studio Code as IDE
- Jupyter Notebook for interactive development
- Python 3.9.21
- GitHub for version control
- Weights & Biases for experiment tracking and hyperparameter optimization
- Claude 3.7 Sonnet for troubleshooting, finding bugs and discussing ideas
- Github Copilot Chat for troubleshooting and finding bugs